## Library

In [ ]:
from confusion import *
from my_models import *
from geneval import *

In [ ]:
def scan_temperatures(xs, prompt, model, tokenizer, temperatures,
                      n_iter = 8, out_length = 10):
    '''
    generate text for a parametrized prompt.
    xs: temperatures
    my_prompt: a parametrizeable prompt
    returns dict with key = temperature, value = list of generated texts
    '''
    ys = {j: [] for j in xs}
    for j in xs:
        generated_output = f_generate_batch(model, tokenizer, [prompt]*n_iter, out_length,
                                            temperature=temperatures(j))
        print(j, generated_output[0])
        ys[j] = (generated_output)
    return ys

In [ ]:
def logprobs_temperatures(model, tokenizer, prompt, ys, xs, dxs, temperatures):
    '''
    return logprobs for prompts scan
    '''
    res = {}
    for j in xs:
        for jj in dxs:
            print(j,jj)
            f_eval_temp = f_eval_prob_batch(model, tokenizer, [prompt]*len(ys[j]), ys[j],
                                            temperature = temperatures(j+jj))
            for jjj, y_temp in enumerate(f_eval_temp):
                res[j,j+jj, jjj] = y_temp
    return res

## temperature scan

In [ ]:
model, tokenizer = my_pythia(device='cuda')

In [ ]:
# parameters 
n_temps = 16 # number temperatures
n_iter = 256 # batch size
T_max = 2.1 # maximal temperatures

xs = np.arange(1, n_temps)
dxs = np.arange(-1,2,1) # what neighbors to consider

def temperatures(x): return np.maximum(T_max*x/xs[-1], T_max*xs[0]/xs[-1])

In [ ]:
prompt = 'I'

In [ ]:
# generate samples


In [ ]:
# eval samples at respective temperature


In [ ]:
# calculate dissimilarity g with L-parameter L
L=1

Ys = []
for j in range(1000):
    ys = scan_temperatures(xs, prompt, model, tokenizer, temperatures, n_iter = n_iter, out_length=10)
    res = logprobs_temperatures(model, tokenizer, prompt, ys, xs, dxs, temperatures)
    Y, dY = stat_indicator_scan_final(xs,ys,res,L, g = lambda x: 2*x-1)
    Ys.append(Y)

In [ ]:
# plot
import pylab as plt
plt.plot(0.5*(temperatures(xs[L:])+temperatures(xs[:-L])), np.mean(Ys,axis=0), 'o')
plt.xlabel('T*')
plt.ylabel('dissimilarity')
plt.ylim(0,0.45)